In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model

In [2]:
def Kh(h,t):
    return np.exp(-t**2/h)
def w(h,ts,T):
    vec = Kh(h, np.arange(1,T+1)-ts)
    return vec/np.sum(vec)

In [3]:
def tvdbn_sa(X,h,lam,tol=1e-7):
    tol=1e-7
    p = X.shape[0]
    T = X.shape[1]-1
    h = T/mid
    A0 = np.random.normal(size=(p,p))
    allA = [A0]
    allA.extend([np.zeros((p,p)) for i in range(T)])
    for i in range(p):
        for ts in range(1,T+1):
            allA[ts][i,:]=allA[ts-1][i,:]
            sq_wt = np.sqrt(w(h,ts,T))
            y = X[i,1:]*sq_wt
            Y = X[:,:-1]*sq_wt
            while True:
                prev = np.copy(allA[ts][i,:])
                for j in range(p):
                    Pj = allA[ts][i,:]@Y - allA[ts][i,j]*Y[j,:] - y
                    Sj = (2/T)*np.dot(Pj, Y[j,:])
                    bj = (2/T)*np.dot(Y[j,:],Y[j,:])
                    allA[ts][i,j]= (np.abs(Sj)>lam)*(np.sign(Sj-lam)*lam-Sj)/bj     
                if np.linalg.norm(prev-allA[ts][i,:],np.inf)<tol:
                    break
    _ = allA.pop(0)
    return allA

In [4]:
def tvdbn(X,h,lam):
    p = X.shape[0]
    T = X.shape[1]-1
    allA = []
    allA.extend([np.zeros((p,p)) for i in range(T)])
    for i in range(p):
        for ts in range(1,T+1):
            sq_wt = np.sqrt(w(h,ts,T))
            y = X[i,1:]*sq_wt
            Y = X[:,:-1]*sq_wt
            clf = linear_model.Lasso(alpha=lam/2,fit_intercept=False)
            clf.fit(Y.T, y)
            allA[ts-1][i,:]=clf.coef_
    return allA

### Stationary data

In [5]:
A1 = np.array([[0,1,0],[0,0,1],[1,0,0]]) 
T=7
As=[]
for i in range(T):
    As.append(A1)
X=np.zeros((3,len(As)+1))
X[:,0]=np.random.normal(size=3)
for i in range(len(As)):
    X[:,i+1]= As[i]@X[:,i] + 0.1*np.random.normal(size=3)

In [6]:
lam=0.1
h=2
allA=tvdbn(X,h,lam)
for i in range(len(As)):
    print(f'A_{i+1}')
    print(As[i])
    print(f'A_{i+1} estimate')
    print(allA[i])
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')

A_1
[[0 1 0]
 [0 0 1]
 [1 0 0]]
A_1 estimate
[[ 0.00796336  0.35926725 -0.        ]
 [ 0.         -0.          0.6946455 ]
 [ 0.76302244  0.         -0.        ]]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
A_2
[[0 1 0]
 [0 0 1]
 [1 0 0]]
A_2 estimate
[[ 0.          0.63069727 -0.        ]
 [ 0.         -0.          0.7466979 ]
 [ 0.64891978  0.         -0.        ]]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
A_3
[[0 1 0]
 [0 0 1]
 [1 0 0]]
A_3 estimate
[[-0.          0.72620952  0.        ]
 [ 0.          0.          0.65796963]
 [ 0.68842746 -0.         -0.        ]]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
A_4
[[0 1 0]
 [0 0 1]
 [1 0 0]]
A_4 estimate
[[ 0.          0.65021627  0.        ]
 [ 0.          0.          0.67323105]
 [ 0.7959302   0.         -0.        ]]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
A_5
[[0 1 0]
 [0 0 1]
 [1 0 0]]
A_5 estimate
[[ 0.          0.69832202  0.        ]
 [ 0.          0.          0.75731756]
 [ 0.70461234  0.         -0.        ]]
$$$$$$$$$$$$$$$$$

### Time varying

In [18]:
A1 = np.array([[0,1,0],[0,0,1],[1,0,0]]) 
A2 = np.array([[0,1,0],[0,0.5,0.5],[1,0,0]]) 
A3 = np.array([[0,1,0],[0,0,1],[0.5,0.5,0]]) 
bigA = [A1,A2,A1]

mid=3
As=[]
for i in range(len(bigA)-1):
    for j in range(mid):
        As.append(bigA[i]+(j/mid)*(bigA[i+1]-bigA[i]))
As.append(bigA[-1])
X=np.zeros((3,len(As)+1))
X[:,0]=np.random.normal(size=3)
for i in range(len(As)):
    X[:,i+1]= As[i]@X[:,i] + 0.1*np.random.normal(size=3)

In [22]:
lam=0.05
T = X.shape[1]-1
h=T/mid
allA=tvdbn(X,h,lam)
for i in range(len(As)):
    print(f'A_{i+1}')
    print(As[i])
    print(f'A_{i+1} estimate')
    print(allA[i])
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')

A_1
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
A_1 estimate
[[-0.          0.02976844 -0.        ]
 [-0.          0.          0.60668264]
 [ 0.86151639  0.          0.        ]]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
A_2
[[0.         1.         0.        ]
 [0.         0.16666667 0.83333333]
 [1.         0.         0.        ]]
A_2 estimate
[[-0.          0.65915357 -0.        ]
 [-0.          0.          0.62883371]
 [ 0.76716813 -0.          0.        ]]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
A_3
[[0.         1.         0.        ]
 [0.         0.33333333 0.66666667]
 [1.         0.         0.        ]]
A_3 estimate
[[-0.          0.77696653 -0.        ]
 [ 0.          0.039898    0.54753785]
 [ 0.67529452  0.          0.        ]]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
A_4
[[0.  1.  0. ]
 [0.  0.5 0.5]
 [1.  0.  0. ]]
A_4 estimate
[[0.         0.68126913 0.        ]
 [0.         0.         0.4094491 ]
 [0.69974346 0.         0.        ]]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


### Bad output

In [5]:
A1 = np.array([[0,1,0],[0,0,1],[1,0,0]]) 
A2 = np.array([[0,0,1],[0,1,0],[1,0,0]]) 
As = []
for i in range(4):
    As.append(A1)
    As.append(A2)
X=np.zeros((3,len(As)+1))
X[:,0]=np.random.normal(size=3)
for i in range(len(As)):
    X[:,i+1]= As[i]@X[:,i] + 0.1*np.random.normal(size=3)

In [10]:
lam=0.05
h=2
allA=tvdbn(X,h,lam)
for i in range(len(As)):
    print(f'A_{i+1}')
    print(As[i])
    print(f'A_{i+1} estimate')
    print(allA[i])
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')

A_1
[[0 1 0]
 [0 0 1]
 [1 0 0]]
A_1 estimate
[[-0.         -0.          0.02097648]
 [-0.          0.         -0.        ]
 [ 0.23417376 -0.         -0.        ]]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
A_2
[[0 0 1]
 [0 1 0]
 [1 0 0]]
A_2 estimate
[[-0.         -0.          0.25147049]
 [-0.          0.         -0.        ]
 [ 0.2011685  -0.         -0.        ]]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
A_3
[[0 1 0]
 [0 0 1]
 [1 0 0]]
A_3 estimate
[[-0.         -0.          0.3127015 ]
 [-0.          0.         -0.        ]
 [ 0.32118661 -0.         -0.        ]]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
A_4
[[0 0 1]
 [0 1 0]
 [1 0 0]]
A_4 estimate
[[-0.         -0.          0.33300992]
 [-0.          0.         -0.        ]
 [ 0.37663888 -0.         -0.        ]]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
A_5
[[0 1 0]
 [0 0 1]
 [1 0 0]]
A_5 estimate
[[-0.         -0.          0.34120476]
 [-0.          0.         -0.        ]
 [ 0.40286671 -0.         -0.        ]]
$$$$$$$$$$$$$$$$$